In [ ]:
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
import numpy as np
from scipy.spatial import cKDTree

# Load the coastline shapefile
coastline = gpd.read_file("D:/FOLDER FROM THESIS/THESIS/Data/VektorDanmark/Coast_all.shp")

# Define raster resolution
resolution = 100  # meters

# Define the raster extent (bounds of the coastline shapefile)
bounds = coastline.total_bounds  # (minx, miny, maxx, maxy)
width = int((bounds[2] - bounds[0]) / resolution)
height = int((bounds[3] - bounds[1]) / resolution)

# Create an empty raster array
distance_raster = np.zeros((height, width), dtype=rasterio.float32)

# Extract coordinates from line geometries
coords = np.vstack([np.array(line.xy).T for line in coastline.geometry if line is not None])
tree = cKDTree(coords)

# Calculate distances
for j in range(height):
    for i in range(width):
        # Convert raster pixel to geographic coordinates
        x = bounds[0] + i * resolution + resolution / 2
        y = bounds[3] - j * resolution - resolution / 2

        # Find nearest coastline point and calculate distance
        dist, _ = tree.query([x, y])
        distance_raster[j, i] = dist

# Save the raster
transform = from_origin(bounds[0], bounds[3], resolution, resolution)
with rasterio.open(
    'D:/FOLDER FROM THESIS/THESIS/Processed data/Distance/coast_distance.tif',
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=distance_raster.dtype,
    crs=coastline.crs,
    transform=transform,
) as dst:
    dst.write(distance_raster, 1)

print("Raster of distance to coast has been saved.")